# Part 1 Prompt
```
--- Day 9: Sensor Boost ---
You've just said goodbye to the rebooted rover and left Mars when you receive a faint distress signal coming from the asteroid belt. It must be the Ceres monitoring station!

In order to lock on to the signal, you'll need to boost your sensors. The Elves send up the latest BOOST program - Basic Operation Of System Test.

While BOOST (your puzzle input) is capable of boosting your sensors, for tenuous safety reasons, it refuses to do so until the computer it runs on passes some checks to demonstrate it is a complete Intcode computer.
```

Boo, not another Intcode one

```
Your existing Intcode computer is missing one key feature: it needs support for parameters in relative mode.

Parameters in mode 2, relative mode, behave very similarly to parameters in position mode: the parameter is interpreted as a position. Like position mode, parameters in relative mode can be read from or written to.

The important difference is that relative mode parameters don't count from address 0. Instead, they count from a value called the relative base. The relative base starts at 0.

The address a relative mode parameter refers to is itself plus the current relative base. When the relative base is 0, relative mode parameters and position mode parameters with the same value refer to the same address.
```

It is linking to Day 5, which makes me think that it doesn't need any of the amplifier stuff

```
For example, given a relative base of 50, a relative mode parameter of -7 refers to memory address 50 + -7 = 43.

The relative base is modified with the relative base offset instruction:

Opcode 9 adjusts the relative base by the value of its only parameter. The relative base increases (or decreases, if the value is negative) by the value of the parameter.

For example, if the relative base is 2000, then after the instruction 109,19, the relative base would be 2019. If the next instruction were 204,-34, then the value at address 1985 would be output.
```

Pulling Day 5 code again

I need to change two things: adding another param mode, and adding opcode 9 that changes the relative base

In [15]:
def get_param_mode(param_modes_str, param_num):
    # param_num is 1, 2, or 3
    if param_num <= len(param_modes_str):
        mode_code = param_modes_str[-param_num]
        if mode_code == "0":
            return "position"
        elif mode_code == "1":
            return "immediate"
        else:
            return "relative"
    else:
        # if it's missing from the instruction, it's always "position"
        return "position"

In [27]:
def get_index_or_value(number, position_mode, program_codes, relative_base):
    if position_mode == "immediate":
        return number
    elif position_mode == "position":
        return program_codes[number]
    else:
        return program_codes[relative_base + number]

In [17]:
def get_value_for_input(param_modes, param_number, program_codes, current_position, relative_base):
    """ Now that we have to do this for the input of more than 3 opcodes, making a
    function makes more sense
    """
    input_mode = get_param_mode(param_modes, param_number)
    input_number = program_codes[current_position + param_number]
    actual_value = get_index_or_value(input_number, input_mode, program_codes, relative_base)
    return actual_value

In [24]:
def run_list_program(program_codes):
    # loop through codes in 4-code "programs"
    current_position = 0
    relative_base = 0
    while True:
        opcode_and_param_modes = str(program_codes[current_position])
        # last two digits is opcode
        opcode = int(opcode_and_param_modes[-2:])

        # made the decision to indent as little as possible, short-circuit instead
        if opcode == 99:
            break
        
        param_modes = opcode_and_param_modes[:-2]
        
        # these opcodes have a first parameter
        if opcode in [1,2,4,5,6,7,8,9]:
            input_1 = get_value_for_input(param_modes, 1, program_codes, current_position, relative_base)
        
        # these opcodes have a second parameter
        if opcode in [1,2,5,6,7,8]:
            input_2 = get_value_for_input(param_modes, 2, program_codes, current_position, relative_base)

        if opcode == 1 or opcode == 2:
            """ Given two inputs and a destination index, perform some operation on 
            the two inputs and put the result in the destination index """
            destination_index = program_codes[current_position+3]

            if opcode == 1:
                output = input_1 + input_2
            elif opcode == 2:
                output = input_1 * input_2

            program_codes[destination_index] = output

            # move to the next "program", opcodes 1 and 2 have length 4
            current_position = current_position + 4
        elif opcode == 3:
            """ Here the input comes from prompting the user, then that is copied to
            the destination index """
            user_input = int(input("Enter a number: "))
            destination_index = program_codes[current_position+1]
            program_codes[destination_index] = user_input
            current_position = current_position + 2
        elif opcode == 4:
            """ Given one input (retrieved above), print it """
            print(input_1)
            current_position = current_position + 2
        elif opcode == 5 or opcode == 6:
            """ Based on the first input, conditionally sets current_position to
            the value of the second input """
            if opcode == 5:
                should_jump = input_1 != 0
            elif opcode == 6:
                should_jump = input_1 == 0
                
            if should_jump:
                current_position = input_2
            else:
                current_position = current_position + 3
        elif opcode == 7 or opcode == 8:
            """ Based on a comparison of the first and second inputs, sets
            destination index to 1 or 0 """
            if opcode == 7:
                comparison_true = input_1 < input_2
            elif opcode == 8:
                comparison_true = input_1 == input_2
                
            destination_index = program_codes[current_position + 3]
            if comparison_true:
                program_codes[destination_index] = 1
            else:
                program_codes[destination_index] = 0
                
            current_position = current_position + 4
        elif opcode == 9:
            relative_base += input_1
            current_position = current_position + 2
        else:
            print("Error, encountered opcode", opcode)
            break

(I deleted the old versions)

In [10]:
example1_input = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]

In [11]:
run_list_program(example1_input)

KeyboardInterrupt: 

That seemed like it got stuck in an infinite loop

Looks like my order of params/args was off

In [19]:
example1_input = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]

In [20]:
run_list_program(example1_input)

KeyboardInterrupt: 

Still infinite looping.  Let's walk through those commands to see how they are supposed to work and what's going wrong

`109, 1`

109 means opcode 09 AKA opcode 9, parameter mode 1 AKA immediate mode

With immediate mode, the parameter is just 1, rather than the parameter at index 1 (even though these are the same in this case)

Opcode 9 means...adjust the relative base.  I am currently just setting it to the parameter value

In [22]:
example1_input = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]

In [23]:
run_list_program(example1_input)

KeyboardInterrupt: 

That didn't fix it...

...because I'm not adjusting the current position either...

In [25]:
example1_input = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]

In [26]:
run_list_program(example1_input)

109


IndexError: list index out of range

I'm assuming this is related to this part of the instructions

```
The computer's available memory should be much larger than the initial program. Memory beyond the initial program starts with the value 0 and can be read or written like any other memory. (It is invalid to try to access memory at a negative address, though.)
```

What does "much larger" mean, though?  Adding some print statements

In [29]:
def get_index_or_value(number, position_mode, program_codes, relative_base):
    if position_mode == "immediate":
        return number
    elif position_mode == "position":
        print("atempting to access index", number)
        return program_codes[number]
    else:
        return program_codes[relative_base + number]

In [30]:
example1_input = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
run_list_program(example1_input)

109
atempting to access index 100


IndexError: list index out of range

I guess let's expand the end of the list so it reaches the number specified

In [31]:
index = 10
program_codes = [1,2,3,4,5]

zero_fill_needed = index - len(program_codes)
zero_fill_needed

5

In [32]:
# fairly inefficient, but readable
for _ in range(zero_fill_needed):
    program_codes.append(0)

In [33]:
program_codes

[1, 2, 3, 4, 5, 0, 0, 0, 0, 0]

In [34]:
index = 10
program_codes = [1,2,3,4,5]

zero_fill_needed = index - len(program_codes) + 1
for _ in range(zero_fill_needed):
    program_codes.append(0)
program_codes[index]

0

In [35]:
for _ in range(-1):
    print("this shouldn't print")

In [40]:
def fill_with_zeroes_to_index(program_codes, index):
    # if we're trying to use an index that's "off the end" of the program,
    # extend the program with a bunch of zeroes
    zero_fill_needed = index - len(program_codes) + 1
    for _ in range(zero_fill_needed):
        program_codes.append(0)

In [41]:
def get_index_or_value(number, position_mode, program_codes, relative_base):
    if position_mode == "immediate":
        return number
    else:
        if position_mode == "position":
            index = number
        elif position_mode == "relative":
            index = relative_base + number
            
        fill_with_zeroes_to_index(program_codes, index)
            
        return program_codes[index]

In [42]:
example1_input = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
run_list_program(example1_input)

109


IndexError: list assignment index out of range

Ok, we need to update run_list_program as well

In [43]:
def run_list_program(program_codes):
    # loop through codes in 4-code "programs"
    current_position = 0
    relative_base = 0
    while True:
        opcode_and_param_modes = str(program_codes[current_position])
        # last two digits is opcode
        opcode = int(opcode_and_param_modes[-2:])

        # made the decision to indent as little as possible, short-circuit instead
        if opcode == 99:
            break
        
        param_modes = opcode_and_param_modes[:-2]
        
        # these opcodes have a first parameter
        if opcode in [1,2,4,5,6,7,8,9]:
            input_1 = get_value_for_input(param_modes, 1, program_codes, current_position, relative_base)
        
        # these opcodes have a second parameter
        if opcode in [1,2,5,6,7,8]:
            input_2 = get_value_for_input(param_modes, 2, program_codes, current_position, relative_base)

        if opcode == 1 or opcode == 2:
            """ Given two inputs and a destination index, perform some operation on 
            the two inputs and put the result in the destination index """
            fill_with_zeroes_to_index(program_codes, current_position+3)
            destination_index = program_codes[current_position+3]

            if opcode == 1:
                output = input_1 + input_2
            elif opcode == 2:
                output = input_1 * input_2

            fill_with_zeroes_to_index(program_codes, destination_index)
            program_codes[destination_index] = output

            # move to the next "program", opcodes 1 and 2 have length 4
            current_position = current_position + 4
        elif opcode == 3:
            """ Here the input comes from prompting the user, then that is copied to
            the destination index """
            user_input = int(input("Enter a number: "))
            
            fill_with_zeroes_to_index(program_codes, current_position+1)
            destination_index = program_codes[current_position+1]
            
            fill_with_zeroes_to_index(program_codes, destination_index)
            program_codes[destination_index] = user_input
            
            current_position = current_position + 2
        elif opcode == 4:
            """ Given one input (retrieved above), print it """
            print(input_1)
            current_position = current_position + 2
        elif opcode == 5 or opcode == 6:
            """ Based on the first input, conditionally sets current_position to
            the value of the second input """
            if opcode == 5:
                should_jump = input_1 != 0
            elif opcode == 6:
                should_jump = input_1 == 0
                
            if should_jump:
                current_position = input_2
            else:
                current_position = current_position + 3
        elif opcode == 7 or opcode == 8:
            """ Based on a comparison of the first and second inputs, sets
            destination index to 1 or 0 """
            if opcode == 7:
                comparison_true = input_1 < input_2
            elif opcode == 8:
                comparison_true = input_1 == input_2
                
            fill_with_zeroes_to_index(program_codes, current_position + 3)
            destination_index = program_codes[current_position + 3]
            
            fill_with_zeroes_to_index(program_codes, destination_index)
            if comparison_true:
                program_codes[destination_index] = 1
            else:
                program_codes[destination_index] = 0
                
            current_position = current_position + 4
        elif opcode == 9:
            relative_base += input_1
            current_position = current_position + 2
        else:
            print("Error, encountered opcode", opcode)
            break

In [44]:
example1_input = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
run_list_program(example1_input)

109
1
204
-1
1001
100
1
100
1008
100
16
101
1006
101
0
99


Good, that worked

I still don't really understand what it means by big numbers.  It's not like I have assumptions built in about how long the numbers are, other than that they are valid integers with base 10

In [45]:
example2_input = [1102,34915192,34915192,7,4,7,99,0]
run_list_program(example2_input)

1219070632396864


In [46]:
len("1219070632396864")

16

Yes, that's a 16-digit number

In [47]:
example3_input = [104,1125899906842624,99]
run_list_program(example3_input)

1125899906842624


Looks fine, let's pull in the real input

In [48]:
file_obj = open("input.txt", "r")
content = file_obj.read().strip()
file_obj.close()

In [49]:
content

'1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,1102,1,3,1000,109,988,209,12,9,1000,209,6,209,3,203,0,1008,1000,1,63,1005,63,65,1008,1000,2,63,1005,63,904,1008,1000,0,63,1005,63,58,4,25,104,0,99,4,0,104,0,99,4,17,104,0,99,0,0,1102,521,1,1028,1101,0,33,1011,1101,0,22,1006,1101,28,0,1018,1102,37,1,1008,1102,1,20,1019,1101,0,405,1026,1101,25,0,1015,1101,330,0,1023,1101,0,29,1016,1101,0,560,1025,1101,24,0,1017,1102,516,1,1029,1102,333,1,1022,1102,1,34,1012,1101,0,402,1027,1101,0,1,1021,1102,36,1,1013,1102,30,1,1002,1101,21,0,1000,1102,1,23,1005,1102,39,1,1003,1102,1,32,1007,1102,26,1,1004,1101,565,0,1024,1101,0,0,1020,1101,0,31,1014,1101,27,0,1001,1101,0,38,1009,1101,0,35,1010,109,-3,2102,1,10,63,1008,63,32,63,1005,63,203,4,187,1106,0,207,1001,64,1,64,1002,64,2,64,109,26,21108,40,40,-4,1005,1019,229,4,213,1001,64,1,64,1105,1,229,1002,64,2,64,109,-20,2102,1,-3,63,1008,63,22,63,1005,63,253,1001,64,1,64,1105,1,255,4,235,1002,64,2,64,109,-10,1208,10,39,63,1005,63,277,4,261,1001,64,1,

In [50]:
run_list_program([int(x) for x in content.split(",")])

Enter a number:  1


203
0


So, I think this means that command 203 is not working correctly

203 should mean opcode 3

```
Opcode 3 takes a single integer as input and saves it to the position given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.
```

The parameter mode here is 2

```
Parameters in mode 2, relative mode, behave very similarly to parameters in position mode: the parameter is interpreted as a position. Like position mode, parameters in relative mode can be read from or written to.

The important difference is that relative mode parameters don't count from address 0. Instead, they count from a value called the relative base. The relative base starts at 0.

The address a relative mode parameter refers to is itself plus the current relative base. When the relative base is 0, relative mode parameters and position mode parameters with the same value refer to the same address.

For example, given a relative base of 50, a relative mode parameter of -7 refers to memory address 50 + -7 = 43.
```

Where exactly is 203 occurring?

In [51]:
import re
[m.start() for m in re.finditer('203', content)]

[105, 733]

In [52]:
content[105:120]

'203,0,1008,1000'

In [53]:
content[733:750]

'203,4,187,1106,0,'

Only one of these should actually be running, since it only asks for my input once

In [54]:
def run_list_program(program_codes):
    # loop through codes in 4-code "programs"
    current_position = 0
    relative_base = 0
    while True:
        opcode_and_param_modes = str(program_codes[current_position])
        # last two digits is opcode
        opcode = int(opcode_and_param_modes[-2:])

        # made the decision to indent as little as possible, short-circuit instead
        if opcode == 99:
            break
        
        param_modes = opcode_and_param_modes[:-2]
        
        # these opcodes have a first parameter
        if opcode in [1,2,3,4,5,6,7,8,9]:
            input_1 = get_value_for_input(param_modes, 1, program_codes, current_position, relative_base)
        
        # these opcodes have a second parameter
        if opcode in [1,2,5,6,7,8]:
            input_2 = get_value_for_input(param_modes, 2, program_codes, current_position, relative_base)

        if opcode == 1 or opcode == 2:
            """ Given two inputs and a destination index, perform some operation on 
            the two inputs and put the result in the destination index """
            fill_with_zeroes_to_index(program_codes, current_position+3)
            destination_index = program_codes[current_position+3]

            if opcode == 1:
                output = input_1 + input_2
            elif opcode == 2:
                output = input_1 * input_2

            fill_with_zeroes_to_index(program_codes, destination_index)
            program_codes[destination_index] = output

            # move to the next "program", opcodes 1 and 2 have length 4
            current_position = current_position + 4
        elif opcode == 3:
            """ Here the input comes from prompting the user, then that is copied to
            the destination index """
            user_input = int(input("Enter a number: "))
            
            destination_index = input_1
            
            fill_with_zeroes_to_index(program_codes, destination_index)
            program_codes[destination_index] = user_input
            
            current_position = current_position + 2
        elif opcode == 4:
            """ Given one input (retrieved above), print it """
            print(input_1)
            current_position = current_position + 2
        elif opcode == 5 or opcode == 6:
            """ Based on the first input, conditionally sets current_position to
            the value of the second input """
            if opcode == 5:
                should_jump = input_1 != 0
            elif opcode == 6:
                should_jump = input_1 == 0
                
            if should_jump:
                current_position = input_2
            else:
                current_position = current_position + 3
        elif opcode == 7 or opcode == 8:
            """ Based on a comparison of the first and second inputs, sets
            destination index to 1 or 0 """
            if opcode == 7:
                comparison_true = input_1 < input_2
            elif opcode == 8:
                comparison_true = input_1 == input_2
                
            fill_with_zeroes_to_index(program_codes, current_position + 3)
            destination_index = program_codes[current_position + 3]
            
            fill_with_zeroes_to_index(program_codes, destination_index)
            if comparison_true:
                program_codes[destination_index] = 1
            else:
                program_codes[destination_index] = 0
                
            current_position = current_position + 4
        elif opcode == 9:
            relative_base += input_1
            current_position = current_position + 2
        else:
            print("Error, encountered opcode", opcode)
            break

Ok, the old version of opcode 3 assumed that it was always immediate mode.  Let's do the full check instead

In [55]:
run_list_program([int(x) for x in content.split(",")])

Enter a number:  1


203
0


That didn't fix anything.  Let's add more verbose logging

In [60]:
def run_list_program(program_codes, verbose=False):
    # loop through codes in 4-code "programs"
    current_position = 0
    relative_base = 0
    while True:
        opcode_and_param_modes = str(program_codes[current_position])
        # last two digits is opcode
        opcode = int(opcode_and_param_modes[-2:])

        # made the decision to indent as little as possible, short-circuit instead
        if opcode == 99:
            break
        
        param_modes = opcode_and_param_modes[:-2]
        
        if verbose:
            print("Opcode", opcode, "and param modes", param_modes)
        
        # these opcodes have a first parameter
        if opcode in [1,2,3,4,5,6,7,8,9]:
            input_1 = get_value_for_input(param_modes, 1, program_codes, current_position, relative_base)
            if verbose:
                print("input_1 is", input_1)
        
        # these opcodes have a second parameter
        if opcode in [1,2,5,6,7,8]:
            input_2 = get_value_for_input(param_modes, 2, program_codes, current_position, relative_base)
            if verbose:
                print("input_2 is", input_2)

        if opcode == 1 or opcode == 2:
            """ Given two inputs and a destination index, perform some operation on 
            the two inputs and put the result in the destination index """
#             fill_with_zeroes_to_index(program_codes, current_position+3)
#             destination_index = program_codes[current_position+3]
            destination_index = get_value_for_input(param_modes, 3, program_codes, current_position, relative_base)

            if opcode == 1:
                output = input_1 + input_2
                if verbose:
                    print("Adding inputs to create output", output)
            elif opcode == 2:
                output = input_1 * input_2
                if verbose:
                    print("Multiplying inputs to create output", output)

            fill_with_zeroes_to_index(program_codes, destination_index)
            program_codes[destination_index] = output
            
            if verbose:
                print("Index", destination_index, "is now", output)

            # move to the next "program", opcodes 1 and 2 have length 4
            current_position = current_position + 4
        elif opcode == 3:
            """ Here the input comes from prompting the user, then that is copied to
            the destination index """
            user_input = int(input("Enter a number: "))
            
            destination_index = input_1
            
            fill_with_zeroes_to_index(program_codes, destination_index)
            program_codes[destination_index] = user_input
            if verbose:
                print("Index", destination_index, "is now", user_input)
            
            current_position = current_position + 2
        elif opcode == 4:
            """ Given one input (retrieved above), print it """
            print(input_1)
            current_position = current_position + 2
        elif opcode == 5 or opcode == 6:
            """ Based on the first input, conditionally sets current_position to
            the value of the second input """
            if opcode == 5:
                should_jump = input_1 != 0
                if verbose:
                    print("input_1 != 0 ?", should_jump )
            elif opcode == 6:
                should_jump = input_1 == 0
                if verbose:
                    print("input_1 == 0 ?", should_jump)
            if should_jump:
                current_position = input_2
                if verbose:
                    print("Just jumpted to", input_2)
            else:
                current_position = current_position + 3
        elif opcode == 7 or opcode == 8:
            """ Based on a comparison of the first and second inputs, sets
            destination index to 1 or 0 """
            if opcode == 7:
                comparison_true = input_1 < input_2
                if verbose:
                    print("input_1 < input_2 ?", comparison_true)
            elif opcode == 8:
                comparison_true = input_1 == input_2
                if verbose:
                    print("input_1 == input_2 ?", comparison_true)
                
#             fill_with_zeroes_to_index(program_codes, current_position + 3)
#             destination_index = program_codes[current_position + 3]
            destination_index = get_value_for_input(param_modes, 3, program_codes, current_position, relative_base)
            
            fill_with_zeroes_to_index(program_codes, destination_index)
            if comparison_true:
                program_codes[destination_index] = 1
                if verbose:
                    print("Index", destination_index, "is now 1")
            else:
                program_codes[destination_index] = 0
                if verbose:
                    print("Index", destination_index, "is now 0")
                
            current_position = current_position + 4
        elif opcode == 9:
            relative_base += input_1
            if verbose:
                print("Adding", input_1, "to the relative base")
                print("Relative base is now", relative_base)
            current_position = current_position + 2
        else:
            print("Error, encountered opcode", opcode)
            break

In [61]:
run_list_program([int(x) for x in content.split(",")], verbose=True)

Opcode 2 and param modes 11
input_1 is 34463338
input_2 is 34463338
Multiplying inputs to create output 1187721666102244
Index 0 is now 1187721666102244
Opcode 7 and param modes 10
input_1 is 0
input_2 is 34463338
input_1 < input_2 ? True
Index 0 is now 1
Opcode 5 and param modes 10
input_1 is 0
input_2 is 53
input_1 != 0 ? False
Opcode 2 and param modes 11
input_1 is 1
input_2 is 3
Multiplying inputs to create output 3
Index 0 is now 3
Opcode 9 and param modes 1
input_1 is 988
Adding 988 to the relative base
Relative base is now 988
Opcode 9 and param modes 2
input_1 is 0
Adding 0 to the relative base
Relative base is now 988
Opcode 9 and param modes 
input_1 is 0
Adding 0 to the relative base
Relative base is now 988
Opcode 9 and param modes 2
input_1 is 0
Adding 0 to the relative base
Relative base is now 988
Opcode 9 and param modes 2
input_1 is 0
Adding 0 to the relative base
Relative base is now 988
Opcode 3 and param modes 2
input_1 is 0


Enter a number:  1


Index 0 is now 1
Opcode 8 and param modes 10
input_1 is 0
input_2 is 1
input_1 == input_2 ? False
Index 0 is now 0
Opcode 5 and param modes 10
input_1 is 0
input_2 is 65
input_1 != 0 ? False
Opcode 8 and param modes 10
input_1 is 0
input_2 is 2
input_1 == input_2 ? False
Index 0 is now 0
Opcode 5 and param modes 10
input_1 is 0
input_2 is 904
input_1 != 0 ? False
Opcode 8 and param modes 10
input_1 is 0
input_2 is 0
input_1 == input_2 ? True
Index 0 is now 1
Opcode 5 and param modes 10
input_1 is 0
input_2 is 58
input_1 != 0 ? False
Opcode 4 and param modes 
input_1 is 203
203
Opcode 4 and param modes 1
input_1 is 0
0


1102 means param modes `11` and opcode `2`

The full instruction is `1102,34463338,34463338,63,`

In [63]:
program_codes = [1102,34463338,34463338,63]
run_list_program(program_codes, verbose=True)

Opcode 2 and param modes 11
input_1 is 34463338
input_2 is 34463338
Multiplying inputs to create output 1187721666102244
Index 0 is now 1187721666102244
Opcode 0 and param modes 
Error, encountered opcode 0


Yeah, that's not right, there is no param mode here, which I _thought_ meant that it should automatically be position mode, but it really looks here like I should be making index 63 be equal to this big number, not index 0

Let's make all the `destination_index` variables go back to being immediate mode

In [72]:
def run_list_program(program_codes, verbose=False):
    # loop through codes in 4-code "programs"
    current_position = 0
    relative_base = 0
    while True:
        opcode_and_param_modes = str(program_codes[current_position])
        # last two digits is opcode
        opcode = int(opcode_and_param_modes[-2:])

        # made the decision to indent as little as possible, short-circuit instead
        if opcode == 99:
            break
        
        param_modes = opcode_and_param_modes[:-2]
        
        if verbose:
            print("**********Opcode", opcode, "and param modes", param_modes, "***************")
        
        # these opcodes have a first parameter
        if opcode in [1,2,3,4,5,6,7,8,9]:
            input_1 = get_value_for_input(param_modes, 1, program_codes, current_position, relative_base)
            if verbose:
                print("input_1 is", input_1)
        
        # these opcodes have a second parameter
        if opcode in [1,2,5,6,7,8]:
            input_2 = get_value_for_input(param_modes, 2, program_codes, current_position, relative_base)
            if verbose:
                print("input_2 is", input_2)

        if opcode == 1 or opcode == 2:
            """ Given two inputs and a destination index, perform some operation on 
            the two inputs and put the result in the destination index """
            fill_with_zeroes_to_index(program_codes, current_position+3)
            destination_index = program_codes[current_position+3]
#             destination_index = get_value_for_input(param_modes, 3, program_codes, current_position, relative_base)

            if opcode == 1:
                output = input_1 + input_2
                if verbose:
                    print("Adding inputs to create output", output)
            elif opcode == 2:
                output = input_1 * input_2
                if verbose:
                    print("Multiplying inputs to create output", output)

            fill_with_zeroes_to_index(program_codes, destination_index)
            program_codes[destination_index] = output
            
            if verbose:
                print("Index", destination_index, "is now", output)

            # move to the next "program", opcodes 1 and 2 have length 4
            current_position = current_position + 4
        elif opcode == 3:
            """ Here the input comes from prompting the user, then that is copied to
            the destination index """
            user_input = int(input("Enter a number: "))
            
            destination_index = input_1
            
            fill_with_zeroes_to_index(program_codes, destination_index)
            program_codes[destination_index] = user_input
            if verbose:
                print("Index", destination_index, "is now", user_input)
            
            current_position = current_position + 2
        elif opcode == 4:
            """ Given one input (retrieved above), print it """
            print(input_1)
            current_position = current_position + 2
        elif opcode == 5 or opcode == 6:
            """ Based on the first input, conditionally sets current_position to
            the value of the second input """
            if opcode == 5:
                should_jump = input_1 != 0
                if verbose:
                    print("input_1 != 0 ?", should_jump )
            elif opcode == 6:
                should_jump = input_1 == 0
                if verbose:
                    print("input_1 == 0 ?", should_jump)
            if should_jump:
                current_position = input_2
                if verbose:
                    print("Just jumpted to", input_2)
            else:
                current_position = current_position + 3
        elif opcode == 7 or opcode == 8:
            """ Based on a comparison of the first and second inputs, sets
            destination index to 1 or 0 """
            if opcode == 7:
                comparison_true = input_1 < input_2
                if verbose:
                    print("input_1 < input_2 ?", comparison_true)
            elif opcode == 8:
                comparison_true = input_1 == input_2
                if verbose:
                    print("input_1 == input_2 ?", comparison_true)
                
            fill_with_zeroes_to_index(program_codes, current_position + 3)
            destination_index = program_codes[current_position + 3]
#             destination_index = get_value_for_input(param_modes, 3, program_codes, current_position, relative_base)
            
            fill_with_zeroes_to_index(program_codes, destination_index)
            if comparison_true:
                program_codes[destination_index] = 1
                if verbose:
                    print("Index", destination_index, "is now 1")
            else:
                program_codes[destination_index] = 0
                if verbose:
                    print("Index", destination_index, "is now 0")
                
            current_position = current_position + 4
        elif opcode == 9:
            relative_base += input_1
            if verbose:
                print("Adding", input_1, "to the relative base")
                print("Relative base is now", relative_base)
            current_position = current_position + 2
        else:
            print("Error, encountered opcode", opcode)
            break

In [73]:
program_codes = [1102,34463338,34463338,63,99]
run_list_program(program_codes, verbose=True)

**********Opcode 2 and param modes 11 ***************
input_1 is 34463338
input_2 is 34463338
Multiplying inputs to create output 1187721666102244
Index 63 is now 1187721666102244


In [74]:
program_codes = [1102,34463338,34463338,63,1007,63,34463338,63,99]
run_list_program(program_codes, verbose=True)

**********Opcode 2 and param modes 11 ***************
input_1 is 34463338
input_2 is 34463338
Multiplying inputs to create output 1187721666102244
Index 63 is now 1187721666102244
**********Opcode 7 and param modes 10 ***************
input_1 is 1187721666102244
input_2 is 34463338
input_1 < input_2 ? False
Index 63 is now 0


In [75]:
program_codes = [1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,99]
run_list_program(program_codes, verbose=True)

**********Opcode 2 and param modes 11 ***************
input_1 is 34463338
input_2 is 34463338
Multiplying inputs to create output 1187721666102244
Index 63 is now 1187721666102244
**********Opcode 7 and param modes 10 ***************
input_1 is 1187721666102244
input_2 is 34463338
input_1 < input_2 ? False
Index 63 is now 0
**********Opcode 5 and param modes 10 ***************
input_1 is 0
input_2 is 53
input_1 != 0 ? False


In [76]:
run_list_program([int(x) for x in content.split(",")], verbose=True)

**********Opcode 2 and param modes 11 ***************
input_1 is 34463338
input_2 is 34463338
Multiplying inputs to create output 1187721666102244
Index 63 is now 1187721666102244
**********Opcode 7 and param modes 10 ***************
input_1 is 1187721666102244
input_2 is 34463338
input_1 < input_2 ? False
Index 63 is now 0
**********Opcode 5 and param modes 10 ***************
input_1 is 0
input_2 is 53
input_1 != 0 ? False
**********Opcode 2 and param modes 11 ***************
input_1 is 1
input_2 is 3
Multiplying inputs to create output 3
Index 1000 is now 3
**********Opcode 9 and param modes 1 ***************
input_1 is 988
Adding 988 to the relative base
Relative base is now 988
**********Opcode 9 and param modes 2 ***************
input_1 is 3
Adding 3 to the relative base
Relative base is now 991
**********Opcode 9 and param modes  ***************
input_1 is 3
Adding 3 to the relative base
Relative base is now 994
**********Opcode 9 and param modes 2 ***************
input_1 is 3
Ad

Enter a number:  1


Index 3 is now 1
**********Opcode 8 and param modes 10 ***************
input_1 is 3
input_2 is 1
input_1 == input_2 ? False
Index 63 is now 0
**********Opcode 5 and param modes 10 ***************
input_1 is 0
input_2 is 65
input_1 != 0 ? False
**********Opcode 8 and param modes 10 ***************
input_1 is 3
input_2 is 2
input_1 == input_2 ? False
Index 63 is now 0
**********Opcode 5 and param modes 10 ***************
input_1 is 0
input_2 is 904
input_1 != 0 ? False
**********Opcode 8 and param modes 10 ***************
input_1 is 3
input_2 is 0
input_1 == input_2 ? False
Index 63 is now 0
**********Opcode 5 and param modes 10 ***************
input_1 is 0
input_2 is 58
input_1 != 0 ? False
**********Opcode 4 and param modes  ***************
input_1 is 203
203
**********Opcode 4 and param modes 1 ***************
input_1 is 0
0


It seems to be behaving literally exactly as expected...

...so I went to the subreddit and they are saying that `destination_index` can be relative mode or immediate mode, whereas the input can be any of the three modes

In [77]:
def get_param_mode(param_modes_str, param_num):
    # param_num is 1, 2, or 3
    if param_num <= len(param_modes_str):
        mode_code = param_modes_str[-param_num]
        if mode_code == "0":
            return "position"
        elif mode_code == "1":
            return "immediate"
        else:
            return "relative"
    else:
        # missing is the same as "0"
        return "position"

In [78]:
def get_index_or_value(number, position_mode, program_codes, relative_base, is_input):
    
    if is_input:
        if position_mode == "immediate":
            return number
        elif position_mode == "position":
            return program_codes[number]
        elif position_mode == "relative":
            return program_codes[relative_base + number]
    else:
        if position_mode == "immediate" or position_mode == "position":
            return number
        elif position_mode == "relative":
            return relative_base + number

In [79]:
def get_value_for_input(param_modes, param_number, program_codes, current_position, relative_base, is_input):
    """ Now that we have to do this for the input of more than 3 opcodes, making a
    function makes more sense
    """
    input_mode = get_param_mode(param_modes, param_number)
    input_number = program_codes[current_position + param_number]
    actual_value = get_index_or_value(input_number, input_mode, program_codes, relative_base, is_input)
    return actual_value

In [80]:
def run_list_program(program_codes, verbose=False):
    # loop through codes in 4-code "programs"
    current_position = 0
    relative_base = 0
    while True:
        opcode_and_param_modes = str(program_codes[current_position])
        # last two digits is opcode
        opcode = int(opcode_and_param_modes[-2:])

        # made the decision to indent as little as possible, short-circuit instead
        if opcode == 99:
            break
        
        param_modes = opcode_and_param_modes[:-2]
        
        if verbose:
            print("**********Opcode", opcode, "and param modes", param_modes, "***************")
        
        # these opcodes have a first parameter
        if opcode in [1,2,3,4,5,6,7,8,9]:
            input_1 = get_value_for_input(param_modes, 1, program_codes, current_position, relative_base, True)
            if verbose:
                print("input_1 is", input_1)
        
        # these opcodes have a second parameter
        if opcode in [1,2,5,6,7,8]:
            input_2 = get_value_for_input(param_modes, 2, program_codes, current_position, relative_base, True)
            if verbose:
                print("input_2 is", input_2)

        if opcode == 1 or opcode == 2:
            """ Given two inputs and a destination index, perform some operation on 
            the two inputs and put the result in the destination index """
#             fill_with_zeroes_to_index(program_codes, current_position+3)
#             destination_index = program_codes[current_position+3]
            destination_index = get_value_for_input(param_modes, 3, program_codes, current_position, relative_base, False)

            if opcode == 1:
                output = input_1 + input_2
                if verbose:
                    print("Adding inputs to create output", output)
            elif opcode == 2:
                output = input_1 * input_2
                if verbose:
                    print("Multiplying inputs to create output", output)

            fill_with_zeroes_to_index(program_codes, destination_index)
            program_codes[destination_index] = output
            
            if verbose:
                print("Index", destination_index, "is now", output)

            # move to the next "program", opcodes 1 and 2 have length 4
            current_position = current_position + 4
        elif opcode == 3:
            """ Here the input comes from prompting the user, then that is copied to
            the destination index """
            user_input = int(input("Enter a number: "))
            
            destination_index = input_1
            
            fill_with_zeroes_to_index(program_codes, destination_index)
            program_codes[destination_index] = user_input
            if verbose:
                print("Index", destination_index, "is now", user_input)
            
            current_position = current_position + 2
        elif opcode == 4:
            """ Given one input (retrieved above), print it """
            print(input_1)
            current_position = current_position + 2
        elif opcode == 5 or opcode == 6:
            """ Based on the first input, conditionally sets current_position to
            the value of the second input """
            if opcode == 5:
                should_jump = input_1 != 0
                if verbose:
                    print("input_1 != 0 ?", should_jump )
            elif opcode == 6:
                should_jump = input_1 == 0
                if verbose:
                    print("input_1 == 0 ?", should_jump)
            if should_jump:
                current_position = input_2
                if verbose:
                    print("Just jumpted to", input_2)
            else:
                current_position = current_position + 3
        elif opcode == 7 or opcode == 8:
            """ Based on a comparison of the first and second inputs, sets
            destination index to 1 or 0 """
            if opcode == 7:
                comparison_true = input_1 < input_2
                if verbose:
                    print("input_1 < input_2 ?", comparison_true)
            elif opcode == 8:
                comparison_true = input_1 == input_2
                if verbose:
                    print("input_1 == input_2 ?", comparison_true)
                
#             fill_with_zeroes_to_index(program_codes, current_position + 3)
#             destination_index = program_codes[current_position + 3]
            destination_index = get_value_for_input(param_modes, 3, program_codes, current_position, relative_base, False)
            
            fill_with_zeroes_to_index(program_codes, destination_index)
            if comparison_true:
                program_codes[destination_index] = 1
                if verbose:
                    print("Index", destination_index, "is now 1")
            else:
                program_codes[destination_index] = 0
                if verbose:
                    print("Index", destination_index, "is now 0")
                
            current_position = current_position + 4
        elif opcode == 9:
            relative_base += input_1
            if verbose:
                print("Adding", input_1, "to the relative base")
                print("Relative base is now", relative_base)
            current_position = current_position + 2
        else:
            print("Error, encountered opcode", opcode)
            break

In [81]:
run_list_program([int(x) for x in content.split(",")], verbose=True)

**********Opcode 2 and param modes 11 ***************
input_1 is 34463338
input_2 is 34463338
Multiplying inputs to create output 1187721666102244
Index 63 is now 1187721666102244
**********Opcode 7 and param modes 10 ***************
input_1 is 1187721666102244
input_2 is 34463338
input_1 < input_2 ? False
Index 63 is now 0
**********Opcode 5 and param modes 10 ***************
input_1 is 0
input_2 is 53
input_1 != 0 ? False
**********Opcode 2 and param modes 11 ***************
input_1 is 1
input_2 is 3
Multiplying inputs to create output 3
Index 1000 is now 3
**********Opcode 9 and param modes 1 ***************
input_1 is 988
Adding 988 to the relative base
Relative base is now 988
**********Opcode 9 and param modes 2 ***************
input_1 is 3
Adding 3 to the relative base
Relative base is now 991
**********Opcode 9 and param modes  ***************
input_1 is 3
Adding 3 to the relative base
Relative base is now 994
**********Opcode 9 and param modes 2 ***************
input_1 is 3
Ad

Enter a number:  1


Index 3 is now 1
**********Opcode 8 and param modes 10 ***************
input_1 is 3
input_2 is 1
input_1 == input_2 ? False
Index 63 is now 0
**********Opcode 5 and param modes 10 ***************
input_1 is 0
input_2 is 65
input_1 != 0 ? False
**********Opcode 8 and param modes 10 ***************
input_1 is 3
input_2 is 2
input_1 == input_2 ? False
Index 63 is now 0
**********Opcode 5 and param modes 10 ***************
input_1 is 0
input_2 is 904
input_1 != 0 ? False
**********Opcode 8 and param modes 10 ***************
input_1 is 3
input_2 is 0
input_1 == input_2 ? False
Index 63 is now 0
**********Opcode 5 and param modes 10 ***************
input_1 is 0
input_2 is 58
input_1 != 0 ? False
**********Opcode 4 and param modes  ***************
input_1 is 203
203
**********Opcode 4 and param modes 1 ***************
input_1 is 0
0


That is literally just the same error.  I think I'm going to re-write this so it's structured differently, because there's too much going on to isolate what's going wrong here

Structurally, I need three global variables:

 - program codes
 - current position
 - relative base

Everything else can be derived from those

I think I am going to make an outer loop that contains those things, then a function for each opcode, only extracting helper functions as the opcodes get really repetitive

In [82]:
def get_param_mode(param_modes_str, param_num):
    # param_num is 1, 2, or 3
    if param_num <= len(param_modes_str):
        mode_code = param_modes_str[-param_num]
        if mode_code == "0":
            return "position"
        elif mode_code == "1":
            return "immediate"
        elif mode_code == "2":
            return "relative"
    else:
        # missing is the same as "0"
        return "position"

In [103]:
def fill_with_zeroes_to_index(program_codes, index):
    # if we're trying to use an index that's "off the end" of the program,
    # extend the program with a bunch of zeroes
    zero_fill_needed = index - len(program_codes) + 1
    for _ in range(zero_fill_needed):
        program_codes.append(0)

In [104]:
def get_input_value(program_codes, param_modes, current_position, relative_base, param_num):
    param_mode = get_param_mode(param_modes, param_num)
    if param_mode == "position":
        fill_with_zeroes_to_index(program_codes, current_position + param_num)
        input_index = program_codes[current_position + param_num]
        
        fill_with_zeroes_to_index(program_codes, input_index)
        input_value = program_codes[input_index]
    elif param_mode == "immediate":
        fill_with_zeroes_to_index(program_codes, current_position + param_num)
        input_value = program_codes[current_position + param_num]
    elif param_mode == "relative":
        fill_with_zeroes_to_index(program_codes, current_position + param_num)
        input_index = program_codes[current_position + param_num] + relative_base
        
        fill_with_zeroes_to_index(program_codes, input_index)
        input_value = program_codes[input_index]
        
    return input_value

In [105]:
def get_destination_index(program_codes, param_modes, current_position, relative_base, param_num):
    param_mode = get_param_mode(param_modes, param_num)
    
    if param_mode == "immediate" or param_mode == "position":
        fill_with_zeroes_to_index(program_codes, current_position + param_num)
        destination_index = program_codes[current_position + param_num]
    elif param_mode == "relative":
        fill_with_zeroes_to_index(program_codes, current_position + param_num)
        destination_index = program_codes[current_position + param_num] + relative_base
        
    return destination_index

In [117]:
def run_opcode_1(program_codes, param_modes, current_position, relative_base):
    """
    Opcode 1: take in two inputs, add them together, set the destination index to the result
    """
    first_input = get_input_value(program_codes, param_modes, current_position, relative_base, 1)
    second_input = get_input_value(program_codes, param_modes, current_position, relative_base, 2)
    
    destination_index = get_destination_index(program_codes, param_modes, current_position, relative_base, 3)
    
    output = first_input + second_input
    
    fill_with_zeroes_to_index(program_codes, destination_index)
    program_codes[destination_index] = output
    
    current_position += 4
    
    return current_position, relative_base

In [118]:
def run_opcode_2(program_codes, param_modes, current_position, relative_base):
    """
    Opcode 2: take in two inputs, multiply them together, set the destination index to the result
    """
    first_input = get_input_value(program_codes, param_modes, current_position, relative_base, 1)
    second_input = get_input_value(program_codes, param_modes, current_position, relative_base, 2)
    
    destination_index = get_destination_index(program_codes, param_modes, current_position, relative_base, 3)
    
    output = first_input * second_input
    
    fill_with_zeroes_to_index(program_codes, destination_index)
    program_codes[destination_index] = output
    
    current_position += 4
    
    return current_position, relative_base

In [108]:
def run_opcode_3(program_codes, param_modes, current_position, relative_base):
    """
    Opcode 3: get input from the user, set the destination index to the result
    """
    user_input = int(input("Enter a number: "))
    destination_index = get_destination_index(program_codes, param_modes, current_position, relative_base, 1)
    
    fill_with_zeroes_to_index(program_codes, destination_index)
    program_codes[destination_index] = user_input
    
    current_position += 2
    
    return current_position, relative_base

In [109]:
def run_opcode_4(program_codes, param_modes, current_position, relative_base):
    """
    Opcode 4: print the value of the 1 and only input
    """
    input_value = get_input_value(program_codes, param_modes, current_position, relative_base, 1)
    
    print(input_value)
    
    current_position += 2
    
    return current_position, relative_base

In [110]:
def run_opcode_5(program_codes, param_modes, current_position, relative_base):
    """
    Opcode 5: if the first input is not 0, set the current position to the second input
    """
    first_input = get_input_value(program_codes, param_modes, current_position, relative_base, 1)
    second_input = get_input_value(program_codes, param_modes, current_position, relative_base, 2)
    
    should_jump = first_input != 0
    
    if should_jump:
        current_position = second_input
    else:
        current_position += 3
        
    return current_position, relative_base
    

In [111]:
def run_opcode_6(program_codes, param_modes, current_position, relative_base):
    """
    Opcode 6: if the first input is 0, set the current position to the second input
    """
    first_input = get_input_value(program_codes, param_modes, current_position, relative_base, 1)
    second_input = get_input_value(program_codes, param_modes, current_position, relative_base, 2)
    
    should_jump = first_input == 0
    
    if should_jump:
        current_position = second_input
    else:
        current_position += 3
        
    return current_position, relative_base

In [112]:
def run_opcode_7(program_codes, param_modes, current_position, relative_base):
    """
    Opcode 7: if the first input is less than the second parameter, store 1 in the
    destination index.  Otherwise, store 0.
    """
    first_input = get_input_value(program_codes, param_modes, current_position, relative_base, 1)
    second_input = get_input_value(program_codes, param_modes, current_position, relative_base, 2)
    
    destination_index = get_destination_index(program_codes, param_modes, current_position, relative_base, 3)
    fill_with_zeroes_to_index(program_codes, destination_index)
    
    comparison_true = first_input < second_input
    
    if comparison_true:
        program_codes[destination_index] = 1
    else:
        program_codes[destination_index] = 0

    current_position = current_position + 4
    
    return current_position, relative_base

In [113]:
def run_opcode_8(program_codes, param_modes, current_position, relative_base):
    """
    Opcode 8: if the first input is equal to the second parameter, store 1 in the
    destination index.  Otherwise, store 0.
    """
    first_input = get_input_value(program_codes, param_modes, current_position, relative_base, 1)
    second_input = get_input_value(program_codes, param_modes, current_position, relative_base, 2)
    
    destination_index = get_destination_index(program_codes, param_modes, current_position, relative_base, 3)
    fill_with_zeroes_to_index(program_codes, destination_index)
    
    comparison_true = first_input == second_input
    
    if comparison_true:
        program_codes[destination_index] = 1
    else:
        program_codes[destination_index] = 0

    current_position = current_position + 4
    
    return current_position, relative_base

In [121]:
def run_opcode_9(program_codes, param_modes, current_position, relative_base):
    """
    Opcode 9: adjusts the relative base by the value of its only parameter.
    The relative base increases (or decreases, if the value is negative)
    by the value of the parameter.
    """
    first_input = get_input_value(program_codes, param_modes, current_position, relative_base, 1)
    relative_base += first_input
    
    current_position += 2
    
    return current_position, relative_base

In [115]:
def run_intcode_program(program_codes, verbose=False):
    current_position = 0
    relative_base = 0
    while True:
        opcode_and_param_modes = str(program_codes[current_position])
        # last two digits is opcode
        opcode = int(opcode_and_param_modes[-2:])

        # made the decision to indent as little as possible, short-circuit instead
        if opcode == 99:
            break
        
        param_modes = opcode_and_param_modes[:-2]
        
        if verbose:
            print("**********Opcode", opcode, "and param modes", param_modes, "***************")
        
        if opcode == 1:
            current_position, relative_base = run_opcode_1(program_codes, param_modes, current_position, relative_base)
        elif opcode == 2:
            current_position, relative_base = run_opcode_2(program_codes, param_modes, current_position, relative_base)
        elif opcode == 3:
            current_position, relative_base = run_opcode_3(program_codes, param_modes, current_position, relative_base)
        elif opcode == 4:
            current_position, relative_base = run_opcode_4(program_codes, param_modes, current_position, relative_base)
        elif opcode == 5:
            current_position, relative_base = run_opcode_5(program_codes, param_modes, current_position, relative_base)
        elif opcode == 6:
            current_position, relative_base = run_opcode_6(program_codes, param_modes, current_position, relative_base)
        elif opcode == 7:
            current_position, relative_base = run_opcode_7(program_codes, param_modes, current_position, relative_base)
        elif opcode == 8:
            current_position, relative_base = run_opcode_8(program_codes, param_modes, current_position, relative_base)
        elif opcode == 9:
            current_position, relative_base = run_opcode_9(program_codes, param_modes, current_position, relative_base)
        else:
            print("Error, encountered opcode", opcode)
            break

In [122]:
run_intcode_program([int(x) for x in content.split(",")], verbose=True)

**********Opcode 2 and param modes 11 ***************
**********Opcode 7 and param modes 10 ***************
**********Opcode 5 and param modes 10 ***************
**********Opcode 2 and param modes 11 ***************
**********Opcode 9 and param modes 1 ***************
**********Opcode 9 and param modes 2 ***************
**********Opcode 9 and param modes  ***************
**********Opcode 9 and param modes 2 ***************
**********Opcode 9 and param modes 2 ***************
**********Opcode 3 and param modes 2 ***************


Enter a number:  1


**********Opcode 8 and param modes 10 ***************
**********Opcode 5 and param modes 10 ***************
**********Opcode 2 and param modes 11 ***************
**********Opcode 1 and param modes 11 ***************
**********Opcode 1 and param modes 11 ***************
**********Opcode 1 and param modes 11 ***************
**********Opcode 2 and param modes 11 ***************
**********Opcode 2 and param modes 11 ***************
**********Opcode 1 and param modes 11 ***************
**********Opcode 1 and param modes 11 ***************
**********Opcode 1 and param modes 11 ***************
**********Opcode 1 and param modes 11 ***************
**********Opcode 1 and param modes 11 ***************
**********Opcode 1 and param modes 11 ***************
**********Opcode 2 and param modes 11 ***************
**********Opcode 2 and param modes 11 ***************
**********Opcode 2 and param modes 11 ***************
**********Opcode 1 and param modes 11 ***************
**********Opcode 1 and param

# Part 2 Prompt
```
--- Part Two ---
You now have a complete Intcode computer.

Finally, you can lock on to the Ceres distress signal! You just need to boost your sensors using the BOOST program.

The program runs in sensor boost mode by providing the input instruction the value 2. Once run, it will boost the sensors automatically, but it might take a few seconds to complete the operation on slower hardware. In sensor boost mode, the program will output a single value: the coordinates of the distress signal.

Run the BOOST program in sensor boost mode. What are the coordinates of the distress signal?
```

I will be pretty happy if this is truly the last Intcode one, even thought it will be somewhat wasted time that I just rewrote all the code from scratch lol

In [123]:
run_intcode_program([int(x) for x in content.split(",")])

Enter a number:  2


58879


...pretty confused why they didn't have opcode 9 be part 1, and zero padding be part 2, but I'm okay with this just working as is